# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="example_opf.png" width="50%">

We first create this network in pandapower:

In [1515]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization:

In [1516]:
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

We run an OPF:

In [1517]:
pp.runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Let's check the results:

In [1518]:
net.res_ext_grid

,p_mw,q_mvar
0,56.530584,1.974564


In [1519]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,71.309255,-1.969681,-3.713031,1.000008
1,12.303443,-1.451180,-3.712735,1.000010


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [1520]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

And now run an OPF:

In [1521]:
pp.runopp(net, delta=1e-16)

We can see that all active power is provided by the external grid: 

In [1522]:
net.res_ext_grid

,p_mw,q_mvar
0,144.559166,9.193021


In [1523]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000079,8.601766,-16.426835,0.967619
1,0.000225,10.594623,-13.481007,0.989756


This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [1524]:
net.res_cost

1445.5955448594816

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [1525]:
net.res_trafo.loading_percent

0    144.851179
Name: loading_percent, dtype: float64

We now limit the transformer loading to 80%:

In [1526]:
net.trafo["max_loading_percent"] =50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [1527]:
pp.runopp(net, delta=1e-16)

We can see that the transformer complies with the maximum loading:

In [1528]:
net.res_trafo.loading_percent

0    49.999136
Name: loading_percent, dtype: float64

And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [1529]:
net.res_ext_grid

,p_mw,q_mvar
0,49.953012,-2.147126


In [1530]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000598,2.992989,-6.232710,0.985230
1,93.304317,3.453173,-1.237784,1.025709


This comes of course with an increase in dispatch costs:

In [1531]:
net.res_cost

1619.1908981410581

### Line Loading Constraints

Wen now look at the line loadings:

In [1532]:
net.res_line.loading_percent

0    22.192863
1    57.476322
2    33.473473
Name: loading_percent, dtype: float64

and run the OPF with a 30% loading constraint:

In [1533]:
net.line["max_loading_percent"] = 30
pp.runopp(net, delta=1e-16)

Now the line loading constraint is complied with:

In [1534]:
net.res_line.loading_percent

0     9.939605
1    29.999977
2    21.140436
Name: loading_percent, dtype: float64

# Locational Marginal Cost

And all generators are involved in supplying the loads:

In [1535]:
net.res_ext_grid



,p_mw,q_mvar
0,49.928571,-2.671419


In [1536]:
pre_power_extg = net.res_ext_grid.p_mw[0]
print (pre_power_extg)

49.92857118797809


In [1537]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,34.729013,0.200389,-4.674661,0.997714
1,56.352451,1.987438,-2.088205,1.019264


In [1538]:
pre_power_g1 = net.res_gen.p_mw[0]
pre_power_g2 = net.res_gen.p_mw[1]
print (pre_power_g1)
print (pre_power_g2)

34.72901264038191
56.35245138314592


In [1539]:
pp.create_load(net, bus2, p_mw=1, controllable=False)

3

Then check the generation units

In [1540]:
pp.runopp(net, delta=1e-16)
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,35.075106,0.192909,-4.625402,0.998116
1,57.014701,2.004789,-2.039977,1.019666


In [1541]:
net.res_ext_grid

,p_mw,q_mvar
0,49.92887,-2.665825


In [1542]:
post_power_extg = net.res_ext_grid.p_mw[0]
post_power_g1 = net.res_gen.p_mw[0]
post_power_g2 = net.res_gen.p_mw[1]
print (post_power_extg)
print (post_power_g1)
print (post_power_g2)

49.928869501427975
35.075106079750235
57.014701472131016


Find the incremental of the power generation of each unit

In [1543]:
incre_Extg = post_power_extg-pre_power_extg
incre_G1 = post_power_g1-pre_power_g1
incre_G2 = post_power_g2-pre_power_g2
print("External Grid",incre_Extg )
print("G1", incre_G1)
print("G2", incre_G2)

External Grid 0.0002983134498819595
G1 0.3460934393683246
G2 0.6622500889850969


In [1544]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,8.189331,-2.588611,-8.112385,0.418652,0.076946,-2.169959,0.044913,0.042716,0.044913,1.003707,-3.423525,0.998116,-4.625402,9.555864
1,-26.812509,-0.225743,27.391297,-0.244024,0.578787,-0.469767,0.141000,0.141000,0.141000,0.998116,-4.625402,1.019666,-2.039977,29.999976
2,19.623405,2.248813,-19.380547,-3.097383,0.242858,-0.848570,0.101671,0.102632,0.102632,1.019666,-2.039977,1.003707,-3.423525,21.836573


Now we assign a cost of 20 ct/kW for the external grid, 15 ct/kw for the generator g0 and 10 ct/kw for generator g1:

In [1545]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 20
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 10

In [1546]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,35.075106,0.192909,-4.625402,0.998116
1,57.014701,2.004789,-2.039977,1.019666


In [1547]:
net.res_ext_grid

,p_mw,q_mvar
0,49.92887,-2.665825


This of course comes with a once again rising dispatch cost:

In [1548]:
net.res_cost

1709.5917038761056

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [1549]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-49.928870,2.665825,10.000000,-2.330340e-22
1,1.003707,-3.423525,61.000000,0.000000,13.146671,-2.819170e-02
2,0.998116,-4.625402,34.924894,-0.192909,15.000000,1.639187e-23
3,1.019666,-2.039977,-47.014701,-2.004789,12.000000,1.757846e-22


and constrain it:

In [1550]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net, delta=1e-16)

We can see that all voltages are within the voltage band:

In [1551]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-37.706781,2.243649,20.000000,-3.202368e-22
1,1.002749,-2.587175,61.000000,0.000000,20.049977,1.612584e-01
2,1.005012,-3.232724,30.065224,-4.981438,15.000000,7.575579e-22
3,1.019999,-0.524516,-54.509029,4.179945,10.000001,-6.276493e-22


And all generators are once again involved in supplying the loads:

In [1552]:
net.res_ext_grid

,p_mw,q_mvar
0,37.706781,-2.243649


In [1553]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,39.934776,4.981438,-3.232724,1.005012
1,64.509029,-4.179945,-0.524516,1.019999


This of course comes once again with rising dispatch costs:

In [1554]:
net.res_cost

1998.2475502685036

## DC OPF

pandapower also provides the possibility of running a DC Optimal Power Flow:

In [1555]:
pp.rundcopp(net, delta=1e-16)

Since voltage magnitudes are not included in the DC power flow formulation, voltage constraints cannot be considered in the DC OPF:

In [1556]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-37.973621,2.243649,20.0,0.0
1,1.002749,-2.610282,61.000000,0.000000,20.0,0.0
2,1.005012,-3.131829,30.701835,-4.981438,15.0,0.0
3,1.019999,-0.524093,-53.728214,4.179945,10.0,0.0


Line and transformer loading limits are however complied with:

In [1557]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,3.837729,0.0,-3.837729,0.0,0.0,0.0,0.020143,0.020143,0.020143,1.0,-2.610282,1.0,-3.131829,4.285713
1,-26.864106,0.0,26.864106,0.0,0.0,0.0,0.141000,0.141000,0.141000,1.0,-3.131829,1.0,-0.524093,29.999998
2,26.864108,0.0,-26.864108,0.0,0.0,0.0,0.141000,0.141000,0.141000,1.0,-0.524093,1.0,-2.610282,30.000000


In [1558]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,37.973621,0.0,-37.973621,0.0,0.0,0.0,0.099655,0.19931,1.0,0.0,1.0,-2.610282,37.973621


As are generator limits:

In [1559]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,min_p_mw,max_p_mw,power_station_trafo
0,None,2,80.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,80.0,NaN
1,None,3,100.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,100.0,NaN


In [1560]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,39.298165,4.981438,-3.131829,1.0
1,63.728214,-4.179945,-0.524093,1.0


The cost function is the same for the linearized OPF as for the non-linear one:

In [1561]:
net.res_cost

1986.2270327733008

## Piecewise linear cost functions

The OPF also offers piecewise linear cost functions. Let us first check the actual cost function setup:

In [1562]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,15.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,10.0,0.0,0.0,0.0,0.0


An element can either have polynomial costs or piecewise linear costs at the same time. So let us first delete the polynomial costs in order to avoid confusion and errors:

In [1563]:
net.poly_cost.drop(net.poly_cost.index.values, inplace=True)

The results above have been produced with linear polynomial cost functions. Let's try to reproduce the results using piecewise linear cost functions. Costs have to be defined for the whole range of generators and external grids:

In [1564]:
net.gen[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,0.0,80.0
1,0.0,100.0


In [1565]:
net.ext_grid[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,-1000.0,1000.0


We define the piecewise linear cost as constant over the whole range to reproduce the polyomial costs defined above:

In [1566]:
pp.create_pwl_cost(net, 0, "gen", [[0, 80, 15]])
pp.create_pwl_cost(net, 1, "gen", [[0, 100, 12]])
pp.create_pwl_cost(net, 0, "ext_grid", [[-1000, 1000, 10]])

2

Let us check the results from the previous OPF again!

In [1567]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,39.298165,4.981438,-3.131829,1.0
1,63.728214,-4.179945,-0.524093,1.0


In [1568]:
net.res_cost

1986.2270327733008

We run the same OPF now with different cost function setup. We should get the exact same results:

In [1569]:
pp.rundcopp(net, delta=1e-16)

In [1570]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,34.924934,4.981438,-4.552835,1.0
1,56.075066,-4.179945,-1.945100,1.0


In [1571]:
net.res_cost

1696.774803977105

Now lets define real piecewise linear costs for generator 1. We define the costs as 12€/MW up to 70MW and 20€/MW from 70MW to 100MW:

In [1572]:
net.pwl_cost.points.loc[1] = [[0, 70, 12], [70, 100, 20]]

And run another OPF:

In [1573]:
pp.rundcopp(net, delta=1e-16)

Now we can see that generator 1 only dispatches 70MW, above which generator 0 becomes less expensive and is therefore dispatched:

In [1574]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,34.924937,4.981438,-4.552835,1.0
1,56.075065,-4.179945,-1.945100,1.0


## Debugging

For more information on the status of the OPF solver, set verbose=True:

In [1575]:
pp.runopp(net, verbose=True, delta=1e-16)

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.89 seconds
Objective Function Value = 1709.75 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              4     Total Gen Capacity    1180.0       -3000000000.0 to 3000000000.0
Generators         3     On-line Capacity      1180.0       -3000000000.0 to 3000000000.0
Committed Gens     3     Generation (actual)    142.0              -0.5
Loads              3     Load                   141.0               0.0
  Fixed            3       Fixed                141.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           4   